In [1]:
from fastai.vision.all import *
from fastai.vision.widgets import *
import chess
from chess import svg
from utils import *
from cairosvg import svg2png

In [2]:
learner = load_learner('latest.pkl')
classes = learner.dls.vocab

In [3]:
def predict_board(img):
    h,w = img.shape
    item_list = []
    for y in range(8):
        for x in range(8):
            box = (x*w/8, y*h/8, (x+1)*w/8, (y+1)*h/8) # left, top, right, bottom
            cropped = img.crop(box)
            square = PILImage.create(cropped.to_bytes_format())
            item_list.append(square) 

    dl = learner.dls.test_dl(item_list)
    _, __, preds = learner.get_preds(dl=dl, with_decoded=True)
    preds = [classes[p] for p in preds]
    labels = ''.join([label_lib[p] for p in preds])
    fen = label2fen(labels)
    
    return fen

In [4]:
def make_png(fen):
    board = chess.Board(fen)
    board_svg = svg.board(board)
    png = svg2png(bytestring=board_svg)
    return png

In [5]:
uploader = widgets.FileUpload()
lbl_fen = widgets.Label()
btn_run = widgets.Button(description='Recognize position')
board_img = widgets.Image(format='png', height=300, width=300)
file = open('starting_position.png', 'rb')
image = file.read()
board_img.value = image

In [6]:
def on_click(change):
    img = Image.open(uploader.data[-1])
    fen = predict_board(img)
    lbl_fen.value = fen
    board_img.value = make_png(fen)

btn_run.on_click(on_click)

In [7]:
VBox([widgets.Label('Upload chess diagram'), 
      uploader, btn_run, lbl_fen, board_img])